In [2]:
!mkdir Scripts

In [10]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
import os

In [18]:
data = ''
for f in os.listdir('/content/Scripts'):
    try:
      with open('/content/Scripts/'+f,"r+") as file:
        content=file.read()
        data+=content
    except:
      pass

In [20]:
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
  in_text = seed_text

  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
    yhat = model.predict_classes(encoded, verbose=0)
    out_word = ''
    for word, index in tokenizer.word_index.items():
      if index == yhat:
        out_word == word
        break
    in_text += ' ' + out_word
  return in_text

In [21]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

In [22]:
vocab_size = len(tokenizer.word_index)+1

In [23]:
sequences=list()
for line in data.split('.'):
  encoded = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)

In [24]:
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)
# split into input and output elements
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length-1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X, y, epochs=10, verbose=2)

Max Sequence Length: 173
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 172, 10)           181660    
                                                                 
 lstm (LSTM)                 (None, 50)                12200     
                                                                 
 dense (Dense)               (None, 18166)             926466    
                                                                 
Total params: 1120326 (4.27 MB)
Trainable params: 1120326 (4.27 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/10
10372/10372 - 612s - loss: 6.9256 - accuracy: 0.0714 - 612s/epoch - 59ms/step
Epoch 2/10
10372/10372 - 611s - loss: 6.1551 - accuracy: 0.1187 - 611s/epoch - 59ms/step
Epoch 3/10
10372/10372 - 611s - loss: 5.8342 - accuracy: 0.1408 - 611s/epo

In [25]:

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
